In [ ]:
!pip install torchaudio
!pip install pydub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torch.utils import data
import torchaudio
import os
from pydub import AudioSegment
import numpy as np
import pydub
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.nn import *
from torch.optim import *
from tqdm import tqdm
import copy
from sklearn.metrics import classification_report
from torch import nn
import torch.nn.functional as F

/usr/local/lib/python3.6/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [ ]:
os.chdir("/content/drive/My Drive")

!cp Audio_Pad.zip /content
!cp Eklenti.zip /content
!cp AudioSet.zip /content

os.chdir("/content")

!unzip Audio_Pad.zip
!unzip Eklenti.zip
!unzip AudioSet.zip

In [ ]:
!mv -v Eklenti/Train/Speech/* Audio_Pad/Train/Speech
!mv -v Eklenti/Train/Non_Speech/* Audio_Pad/Train/Non_Speech
!mv -v Eklenti/Val/Speech/* Audio_Pad/Val/Speech
!mv -v Eklenti/Val/Non_Speech/* Audio_Pad/Val/Non_Speech

mv: cannot stat 'Eklenti/Train/Speech/*': No such file or directory
mv: cannot stat 'Eklenti/Train/Non_Speech/*': No such file or directory
mv: cannot stat 'Eklenti/Val/Speech/*': No such file or directory
mv: cannot stat 'Eklenti/Val/Non_Speech/*': No such file or directory


In [ ]:
sp_path = "Audio_Pad/Train/Speech"

nonsp_path = "Audio_Pad/Train/Non_Speech"

sp_tensor = torch.empty((len(os.listdir(sp_path)),128, 157))
nonsp_tensor = torch.empty((len(os.listdir(nonsp_path)),128, 157))

i = 0

for song in os.listdir(sp_path):
    sound, _ = torchaudio.load(sp_path + "/" + song, channels_first = False)
    sound = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128, n_fft = 2048)(sound.flatten())
    sound = torchaudio.transforms.FrequencyMasking(freq_mask_param = 15)(sound)
    sound = torchaudio.transforms.TimeMasking(time_mask_param = 35)(sound)
    
    sp_tensor[i] = sound
    i += 1

i = 0
for song in os.listdir(nonsp_path):
    sound, _ = torchaudio.load(nonsp_path + "/" + song, channels_first = False)
    sound = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128, n_fft = 2048)(sound.flatten())
    sound = torchaudio.transforms.FrequencyMasking(freq_mask_param = 15)(sound)
    sound = torchaudio.transforms.TimeMasking(time_mask_param = 35)(sound)
    
    nonsp_tensor[i] = sound
    i += 1
    
train_data = torch.cat((sp_tensor, nonsp_tensor))

train_labels = torch.cat((torch.ones(len(os.listdir(sp_path))), torch.zeros(len(os.listdir(nonsp_path)))))

sp_tensor = []
nonsp_tensor = []

del sp_tensor
del nonsp_tensor

aug_train_data = torch.zeros((50,128,157))
aug_train_labels = torch.zeros((50))

train_data = torch.cat((train_data, aug_train_data))
train_labels = torch.cat((train_labels,aug_train_labels))

train_data = train_data[:,None, ...]

/usr/local/lib/python3.6/dist-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:653.)
  normalized, onesided, return_complex)
/usr/local/lib/python3.6/dist-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
  normalized, onesided, return_complex)


In [ ]:
sp_path = "Audio_Pad/Val/Speech"

nonsp_path = "Audio_Pad/Val/Non_Speech"

sp_tensor = torch.empty((len(os.listdir(sp_path)),128, 157))
nonsp_tensor = torch.empty((len(os.listdir(nonsp_path)),128, 157))

i = 0

for song in os.listdir(sp_path):
    sound, _ = torchaudio.load(sp_path + "/" + song, channels_first = False)
    sound = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128, n_fft = 2048)(sound.flatten())
    
    sp_tensor[i] = sound
    i += 1

i = 0
for song in os.listdir(nonsp_path):
    sound, _ = torchaudio.load(nonsp_path + "/" + song, channels_first = False)
    sound = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128, n_fft = 2048)(sound.flatten())
    
    nonsp_tensor[i] = sound
    i += 1
    
val_data = torch.cat((sp_tensor, nonsp_tensor))

val_labels = torch.cat((torch.ones(len(os.listdir(sp_path))), torch.zeros(len(os.listdir(nonsp_path)))))

sp_tensor = []
nonsp_tensor = []

del sp_tensor
del nonsp_tensor

val_data = val_data[:,None, ...]
                                                                                              
                                                                                              
sp_path = "Audio_Pad/Test/Speech"

nonsp_path = "Audio_Pad/Test/Non_Speech"

sp_tensor = torch.empty((len(os.listdir(sp_path)),128, 157))
nonsp_tensor = torch.empty((len(os.listdir(nonsp_path)),128, 157))

i = 0

for song in os.listdir(sp_path):
    sound, _ = torchaudio.load(sp_path + "/" + song, channels_first = False)
    sound = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128, n_fft = 2048)(sound.flatten())
    sound = torchaudio.transforms.FrequencyMasking(freq_mask_param = 15)(sound)
    sound = torchaudio.transforms.TimeMasking(time_mask_param=35)(sound)
    
    sp_tensor[i] = sound
    i += 1

i = 0
for song in os.listdir(nonsp_path):
    sound, _ = torchaudio.load(nonsp_path + "/" + song, channels_first = False)
    sound = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128, n_fft = 2048)(sound.flatten())
    sound = torchaudio.transforms.FrequencyMasking(freq_mask_param = 15)(sound)
    sound = torchaudio.transforms.TimeMasking(time_mask_param=35)(sound)
    
    nonsp_tensor[i] = sound
    i += 1
    
test_data = torch.cat((sp_tensor, nonsp_tensor))

test_labels = torch.cat((torch.ones(len(os.listdir(sp_path))), torch.zeros(len(os.listdir(nonsp_path)))))

sp_tensor = []
nonsp_tensor = []

del sp_tensor
del nonsp_tensor

test_data = test_data[:,None, ...]

train_data = torch.cat((train_data, test_data))

train_labels = torch.cat((train_labels, test_labels))

In [ ]:
train_data.shape

torch.Size([10866, 1, 128, 157])

In [ ]:
val_data.shape

torch.Size([4641, 1, 128, 157])

In [ ]:
train_labels.sum()

tensor(5192.)

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, data, label):
        self.data = torch.tensor(data, dtype=torch.float)
        self.label = torch.tensor(label, dtype=torch.long)
        

    def __len__(self):
        return len(self.label)

    def __getitem__(self, index):

        item_data = self.data[index]
        item_label = self.label[index]

        return item_data, item_label

training_set = CustomDataset(train_data, train_labels)
training_generator = DataLoader(training_set, batch_size = 128, shuffle = True)

val_set = CustomDataset(val_data, val_labels)
val_generator = DataLoader(training_set, batch_size = 128, shuffle = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [ ]:
class CNNLayerNorm(Module):
    """Layer normalization built for cnns input"""
    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = LayerNorm(n_feats)

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous() # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous() # (batch, channel, feature, time) 

class BidirectionalGRU(nn.Module):

    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x

class SoundDetectorModel(Module):   
    def __init__(self):
        super(SoundDetectorModel, self).__init__()
          

        self.sound_detector_model =  Sequential(
            Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            CNNLayerNorm(128),
            Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            CNNLayerNorm(128),
            MaxPool2d(kernel_size=2, stride=2),
            Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            CNNLayerNorm(64),
            Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            CNNLayerNorm(64),
            MaxPool2d(kernel_size=2, stride=2),
            Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            Dropout(0.5),
            Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            Dropout(0.5),
            Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            Dropout(0.5)
            )
        
        self.classifier = nn.Sequential(
            ##BidirectionalGRU(1024, 512, 0.1, True),
            ##BidirectionalGRU(1024, 512, 0.1, True),
            Flatten(),
            Linear(16384, 2048), 
            ReLU(inplace=True),
            LayerNorm(2048),
            Linear(2048, 2)
        )
            

    def forward(self, x):

        x = self.sound_detector_model(x)
        
        sizes = x.size()
        x = x.view(sizes[0], sizes[2] * sizes[3], sizes[1])
        ##x = x.transpose(1, 2)
        x = self.classifier(x)

        
        return x


model = SoundDetectorModel()

if torch.cuda.is_available():
    model = model.cuda()

criterion = CrossEntropyLoss()

optimizer = AdamW(model.parameters(), lr=0.0001)

scheduler = lr_scheduler.OneCycleLR(optimizer,
	max_lr=0.0004,
	steps_per_epoch=int(len(training_generator)),
	epochs=50,
	anneal_strategy='linear')

In [ ]:
model(val_data[0][None,...].cuda()).shape

torch.Size([1, 2])

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# number of epochs to train the model
n_epochs = 50

best_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())
val_acc_history = []


for epoch in tqdm(range(1, n_epochs+1)):
    
    for phase in ['train', 'val']:
        
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        # keep track of training and validation loss
        running_loss = 0.0
        running_corrects = 0
 

        for inputs, labels in training_generator:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                _, preds = torch.max(outputs, 1)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()           

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(training_generator.dataset)
        epoch_acc = running_corrects.double() / len(training_generator.dataset)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
    
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        if phase == 'val':
            val_acc_history.append(epoch_acc)



  0%|          | 0/50 [00:00<?, ?it/s]

train Loss: 0.0344 Acc: 0.9894




  2%|▏         | 1/50 [01:02<50:45, 62.15s/it]

val Loss: 0.0408 Acc: 0.9835
train Loss: 0.0323 Acc: 0.9893




  4%|▍         | 2/50 [02:05<49:53, 62.37s/it]

...



 18%|█▊        | 9/50 [09:26<43:05, 63.06s/it]

val Loss: 0.0532 Acc: 0.9827
train Loss: 0.0243 Acc: 0.9924




 20%|██        | 10/50 [10:29<42:02, 63.06s/it]

val Loss: 0.0233 Acc: 0.9934


KeyboardInterrupt: ignored

In [ ]:
def preprocess_audio(data):
    
    if isinstance(data, str):
    
        audio = AudioSegment.from_file(data)
        audio = audio.set_frame_rate(16000)
        audio = audio.set_channels(1)
        audio = audio.get_array_of_samples()
        
        
    elif isinstance(data, np.ndarray) or isinstance(data, torch.Tensor):
        
        audio = np.array(data)
        
        if len(audio.shape) > 2:
            raise ValueError("Ses dizisinin boyutu en fazla 2 olabilir!")
        elif len(audio.shape) == 2:
            if audio.shape[0] == 2:
                torch.reshape((audio.shape[1], audio.shape[0]))
            if isinstance(data, torch.Tensor):
                audio = np.mean(audio.numpy(), axis = 1)
            else:
                audio = np.mean(audio, axis = 1)
            
    aud_tensor = torch.tensor(np.array(audio))
    
    if aud_tensor.shape[0] < 160000:
        pad_shape = 160000 - aud_tensor.shape[0]
        pad_tensor = torch.zeros(pad_shape)
        aud_tensor = torch.cat((aud_tensor, pad_tensor))
        
    elif aud_tensor.shape[0] > 160000:
        splitted_tensor = list(torch.split(aud_tensor, 160000))
        if splitted_tensor[-1].shape[0] < 160000:
            pad_shape = 160000 - splitted_tensor[-1].shape[0]
            if aud_tensor.dtype == torch.int16:
                pad_tensor = torch.zeros(pad_shape, dtype = torch.int16)
            else:
                pad_tensor = torch.zeros(pad_shape)
            splitted_tensor[-1] = torch.cat((splitted_tensor[-1], pad_tensor)) 
        aud_tensor = torch.stack(splitted_tensor)
     
    
    if aud_tensor.dtype == torch.int16:
        aud_tensor = torch.tensor(aud_tensor, dtype = torch.float)
        
    torchaudio.save("deneme.wav", aud_tensor, sample_rate = 16000)
    aud_tensor = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128, n_fft = 2048)(aud_tensor)
    
    if len(aud_tensor.shape) == 2:
        aud_tensor = aud_tensor[None, None, ...]
    elif len(aud_tensor.shape) == 3:
        aud_tensor = aud_tensor[:,None,...]
    
    return aud_tensor

def predict(path):

    song = preprocess_audio(path)
    out = model(song.cuda())

    _, preds = torch.max(out, 1)

    
    if torch.sum(preds) > 0:
        print("Konuşma var!")
    else:
        print("Konuşma yok!")
    
    return torch.sum(preds)

In [ ]:
model.load_state_dict(best_model_wts)

<All keys matched successfully>

In [ ]:
model.eval()

path = "AudioSet/Non_Speech"
len_dir = len(os.listdir(path))
true = 0
results = {}

for song in os.listdir(path):
  with torch.no_grad():
    res = predict(path+ "/" + song)    
    if res == 1:
        print(song)
    else:
        true += 1
        
nonsp_acc = true/len_dir

path = "AudioSet/Speech"
len_dir = len(os.listdir(path))
true = 0
results = {}

for song in os.listdir(path):
  with torch.no_grad():
    res = predict(path+ "/" + song)    
    if res == 0:
        print(song)
    else:
        true += 1
        
sp_acc = true/len_dir
print("Speech Accuracy:",sp_acc,"Non_Speech Accuracy:",nonsp_acc)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Konuşma var!
8021.wav
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma var!
8284.wav
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma var!
8500.wav
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma var!
8522.wav
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma var!
8289.wav
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma var!
7123.wav
Konuşma yok!
Konuşma var!
8506.wav
Konuşma yok!
Konuşma yok!
Konuşma var!
7030.wav
Konuşma yok!
Konuşma var!
7034.wav
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma var!
7015.wav
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma var!
7075.wav
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konuşma yok!
Konu

In [ ]:
torch.save(best_model_wts, "SoundDetector.pth")

torch.save(model,'SoundDetector.pt')